## TP3 : API Github 

Objectif : Identifier le MVP selon son nombre d'étoiles moyen


In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [14]:
#Etape 1 : Création de la liste de contributeurs
url = 'https://gist.github.com/paulmillr/2657075'
soup = BeautifulSoup(requests.get(url).content, "html.parser")

df = pd.DataFrame(columns=['contributors'])
df['contributors'] = [soup.find(string='#'+str(i)).findNext('a').string for i in range(1,256)]

In [15]:
# Récupération du token dans le fichier tkn.txt ou saisie de celui-ci dans le prompt
try :
    with open('tkn.txt', 'r') as tkn :
        token = tkn.readline()[:-1] 
except: 
    token = input('token ?')      

In [59]:
def get_mean_stars(contributor) :
    
    # Récupération du nombre de page
    req_usr_root = requests.get(f'https://api.github.com/users/{contributor}', headers={"Authorization" : f"token {token}"})
    count_public_repos = req_usr_root.json()['public_repos']
    count_pages = 1 + int(count_public_repos/100)
    
    # Calcul du nombre d'étoiles / repo 
    stars = 0
    url_model = 'https://api.github.com/users/{0}/repos?per_page=100&page={1}'
    for page in range(0, count_pages):
        url = url_model.format(contributor, page)
        req_repo = requests.get(url, headers={"Authorization" : f"token {token}"} )
        stars += sum([repo['stargazers_count'] for repo in req_repo.json()]) 
    return stars/count_public_repos if count_public_repos != 0 else 0

In [60]:
# Itérations de la fonction sur les 256 plus gros commiteurs de github (en date)
for i, contributor in enumerate(df['contributors']) :
    df.loc[i, 'Mean stars'] = get_mean_stars(contributor)

In [61]:
# MVP par ordre décroissant d'étoiles / repo
df.sort_values('Mean stars', ascending=False)

,contributors,Mean stars
225,bailicangdu,3503.538462
18,kamranahmedse,2275.477273
109,JakeWharton,2145.980000
142,vinta,1830.406250
117,mrdoob,1597.228571
194,IanLunn,1402.055556
44,sebastianbergmann,1388.800000
230,junyanz,1373.071429
153,nolimits4web,1338.928571
195,ruanyf,1261.622642
